In [2]:
import mediapipe as mp
import cv2
import os
import csv
import math

# Initialize MediaPipe and OpenCV
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

# Set up the CSV file
csv_path = 'test.csv'
fieldnames = ['label'] + [f'dist_{i}_{j}' for i in range(21) for j in range(i+1, 21)] + [f'angle_{i}_{j}_{k}' for i in range(21) for j in range(i+1, 21) for k in range(j+1, 21)]

# Function to calculate the Euclidean distance between two landmarks
def calculate_distance(landmark1, landmark2):
    return math.sqrt((landmark2.x - landmark1.x)**2 + (landmark2.y - landmark1.y)**2 + (landmark2.z - landmark1.z)**2)

# Function to calculate the angle between three landmarks
def calculate_angle(landmark1, landmark2, landmark3):
    vec1 = [landmark2.x - landmark1.x, landmark2.y - landmark1.y, landmark2.z - landmark1.z]
    vec2 = [landmark3.x - landmark2.x, landmark3.y - landmark2.y, landmark3.z - landmark2.z]
    
    dot_product = sum(v1 * v2 for v1, v2 in zip(vec1, vec2))
    magnitude1 = math.sqrt(sum(v**2 for v in vec1))
    magnitude2 = math.sqrt(sum(v**2 for v in vec2))
    
    if magnitude1 == 0 or magnitude2 == 0:
        return 0.0
    
    cosine_angle = dot_product / (magnitude1 * magnitude2)
    angle = math.acos(cosine_angle)
    return math.degrees(angle)

# Open the CSV file in write mode
with open(csv_path, mode='w', newline='') as csvfile:
    csv_writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    csv_writer.writeheader()

    cap = cv2.VideoCapture(0)
    label = 'Thumbs Up'  # Start with the "Thumbs Up" label

    with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands: 
        while cap.isOpened():
            ret, frame = cap.read()
            
            # BGR to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            
            # Flip on horizontal
            image = cv2.flip(image, 1)
            
            # Set flag
            image.flags.writeable = False
            
            # Detections
            results = hands.process(image)
            
            # Set flag to true
            image.flags.writeable = True
            
            # RGB to BGR
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            # Rendering results and collecting landmark data
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS, 
                                            mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                            mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2),
                                             )
                    
                    # Prepare the row data
                    row = {'label': label}
                    
                    # Calculate distances between all pairs of landmarks
                    for i in range(21):
                        for j in range(i+1, 21):
                            distance = calculate_distance(hand_landmarks.landmark[i], hand_landmarks.landmark[j])
                            row[f'dist_{i}_{j}'] = distance
                    
                    # Calculate angles between sets of three landmarks
                    for i in range(21):
                        for j in range(i+1, 21):
                            for k in range(j+1, 21):
                                angle = calculate_angle(hand_landmarks.landmark[i], hand_landmarks.landmark[j], hand_landmarks.landmark[k])
                                row[f'angle_{i}_{j}_{k}'] = angle
                    
                    # Write the row to the CSV file
                    csv_writer.writerow(row)
                    
            # Display the image
            cv2.imshow('Hand Tracking', image)

            # Switch between labels using the keys 'u', 'd', and 'e'
            key = cv2.waitKey(10) & 0xFF
            if key == ord('u'):
                label = 'spider'
            elif key == ord('e'):
                label = 'super'
            elif key == ord('d'):
                label = 'hi'
            elif key == ord('q'):
                break
                
    cap.release()
    cv2.destroyAllWindows()


In [1]:
import pandas as pd

In [7]:
df = pd.read_csv("hand_features.csv")

In [13]:
len(df)

829